Simple Attendance Tracker using Python

In [1]:
import pandas as pd

# Sample data for 10 students
students_data = {
    "Roll Number": [f"00{i+1}" for i in range(10)],
    "Student Mail ID": [f"student{i+1}@example.com" for i in range(10)],
    "Subject 1": ["Math", "English", "Physics", "History", "Biology", "Math", "Chemistry", "Economics", "English", "Geography"],
    "Subject 2": ["Science", "Math", "Chemistry", "Biology", "History", "English", "Physics", "Math", "Biology", "Civics"],
    "Subject 3": ["English", "Science", "Math", "Civics", "Physics", "History", "Biology", "Chemistry", "Economics", "Math"]
}

# Create DataFrame
students_df = pd.DataFrame(students_data)

# Save to Excel
excel_file_path = "Sample_Students_Data.xlsx"
students_df.to_excel(excel_file_path, index=False)

excel_file_path


'Sample_Students_Data.xlsx'

In [3]:
pip install pandas openpyxl


Note: you may need to restart the kernel to use updated packages.


In [5]:
import openpyxl

wb = openpyxl.load_workbook('Sample_Students_Data.xlsx')
sheet = wb.active
print(sheet.cell(row=2, column=1).value)  # Example to print data


001


In [7]:
import pandas as pd
df = pd.read_excel('Sample_Students_Data.xlsx')
df.head()


,Roll Number,Student Mail ID,Subject 1,Subject 2,Subject 3
0,1,student1@example.com,Math,Science,English
1,2,student2@example.com,English,Math,Science
2,3,student3@example.com,Physics,Chemistry,Math
3,4,student4@example.com,History,Biology,Civics
4,5,student5@example.com,Biology,History,Physics


In [9]:
pip install openpyxl ttkbootstrap


Note: you may need to restart the kernel to use updated packages.


In [15]:
import openpyxl
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import tkinter as tk
from tkinter import messagebox
from tkinter import ttk
from getpass import getpass  # For runtime password input (better security)

# ====== CONFIGURATION ======
EXCEL_FILE = 'Sample_Students_Data.xlsx'
FROM_EMAIL = 'crazygirlaks@gmail.com'
EMAIL_PASSWORD = getpass("Enter your email password: ")  # Or use environment variable

STAFF_EMAILS = ['erakshaya485@gmail.com', 'yyyyyyyy@gmail.com']
SUBJECTS = {1: "CI", 2: "Python", 3: "DM"}
WARNINGS = {
    1: "⚠️ Warning: You are allowed only one more absence for CI.",
    2: "⚠️ Warning: You are allowed only one more absence for Python.",
    3: "⚠️ Warning: You are allowed only one more absence for DM."
}

# ====== EXCEL SETUP ======
book = openpyxl.load_workbook(EXCEL_FILE)
sheet = book['Sheet1']


def save_workbook():
    book.save(EXCEL_FILE)


def send_email(recipients, subject, body):
    try:
        with smtplib.SMTP('smtp.gmail.com', 587, timeout=120) as server:
            server.starttls()
            server.login(FROM_EMAIL, EMAIL_PASSWORD)

            for email in recipients:
                msg = MIMEMultipart()
                msg['Subject'] = subject
                msg.attach(MIMEText(body, 'plain'))
                server.sendmail(FROM_EMAIL, email, msg.as_string())
    except Exception as e:
        print(f"[ERROR] Failed to send email: {e}")


def check_attendance_threshold(absences, rows, subject_idx, log_widget):
    warn_list, lack_list, roll_lack_list = [], [], []

    for count, row in zip(absences, rows):
        email = sheet.cell(row=row, column=2).value
        roll_no = sheet.cell(row=row, column=1).value

        if count == 2:
            warn_list.append(email)
        elif count > 2:
            lack_list.append(email)
            roll_lack_list.append(str(roll_no))

    subject_name = SUBJECTS[subject_idx]

    if warn_list:
        send_email(warn_list, f"[{subject_name}] Attendance Warning", WARNINGS[subject_idx])
        log_widget.insert(tk.END, f"⚠️ Warning email sent to: {', '.join(warn_list)}\n")

    if lack_list:
        student_msg = f"🚫 You have insufficient attendance in {subject_name}."
        staff_msg = f"📋 Students lacking attendance in {subject_name}: {', '.join(roll_lack_list)}"
        send_email(lack_list, f"[{subject_name}] Lack of Attendance", student_msg)
        send_email([STAFF_EMAILS[subject_idx - 1]], f"[{subject_name}] Attendance Report", staff_msg)
        log_widget.insert(tk.END, f"🚫 Lack of attendance email sent to students and staff.\n")


def mark_absentees():
    try:
        subject_idx = int(subject_choice.get().split(" - ")[0])
        roll_numbers = list(map(int, entry_rolls.get().strip().split()))

        updated_rows, updated_counts = [], []

        for roll in roll_numbers:
            for i in range(2, sheet.max_row + 1):
                if sheet.cell(row=i, column=1).value == roll:
                    col = subject_idx + 2
                    count = sheet.cell(row=i, column=col).value or 0
                    sheet.cell(row=i, column=col).value = count + 1
                    updated_counts.append(count + 1)
                    updated_rows.append(i)
                    break

        save_workbook()
        log_box.insert(tk.END, f"✅ Marked absent: {', '.join(map(str, roll_numbers))}\n")
        check_attendance_threshold(updated_counts, updated_rows, subject_idx, log_box)

    except Exception as e:
        messagebox.showerror("Error", f"Failed to mark absentees:\n{e}")


def show_absentees():
    try:
        subject_idx = int(subject_choice.get().split(" - ")[0])
        col = subject_idx + 2
        absentees = []

        for i in range(2, sheet.max_row + 1):
            roll = sheet.cell(row=i, column=1).value
            name = sheet.cell(row=i, column=3).value
            count = sheet.cell(row=i, column=col).value or 0
            if count > 0:
                absentees.append(f"{roll} - {name} ({count} absences)")

        subject_name = SUBJECTS[subject_idx]
        log_box.insert(tk.END, f"\n📋 Absentees for {subject_name}:\n" + ("\n".join(absentees) if absentees else "No absentees.\n"))

    except Exception as e:
        messagebox.showerror("Error", f"Failed to show absentees:\n{e}")


root = tk.Tk()
root.title("📊 Student Attendance Manager")
root.geometry("750x600")
root.configure(bg="#f4f4f5")  # Light grayish background

# ====== GUI COMPONENTS ======
tk.Label(
    root,
    text="Student Attendance Management",
    font=("Segoe UI", 18, "bold"),
    bg="#f4f4f5",
    fg="#1e293b"  # Dark slate (almost black-blue)
).pack(pady=15)

frame_controls = tk.Frame(root, bg="#f4f4f5")
frame_controls.pack(pady=10)

tk.Label(
    frame_controls,
    text="Subject",
    font=("Segoe UI", 12),
    bg="#f4f4f5",
    fg="#334155"  # Muted slate
).grid(row=0, column=0, padx=5)
subject_choice = tk.StringVar()
subject_choice.set("1 - CI")
ttk.Combobox(
    frame_controls,
    textvariable=subject_choice,
    values=["1 - CI", "2 - Python", "3 - DM"],
    width=20,
    state="readonly"
).grid(row=0, column=1, padx=5)

tk.Label(
    frame_controls,
    text="Roll Numbers",
    font=("Segoe UI", 12),
    bg="#f4f4f5",
    fg="#334155"
).grid(row=1, column=0, padx=5, pady=10)
entry_rolls = tk.Entry(
    frame_controls,
    font=("Segoe UI", 12),
    width=30,
    bg="white",
    fg="#111827",
    relief="solid",
    borderwidth=1
)
entry_rolls.grid(row=1, column=1, padx=5, pady=10)

btn_frame = tk.Frame(root, bg="#f4f4f5")
btn_frame.pack(pady=10)

tk.Button(
    btn_frame,
    text="✅ Submit Absentees",
    command=mark_absentees,
    font=("Segoe UI", 11, "bold"),
    bg="#3b82f6",  # Blue-500
    fg="white",
    activebackground="#2563eb",
    padx=12,
    pady=6
).pack(side=tk.LEFT, padx=10)

tk.Button(
    btn_frame,
    text="📋 Show Absentees",
    command=show_absentees,
    font=("Segoe UI", 11, "bold"),
    bg="#10b981",  # Emerald-500
    fg="white",
    activebackground="#059669",
    padx=12,
    pady=6
).pack(side=tk.LEFT, padx=10)

log_box = tk.Text(
    root,
    font=("Consolas", 10),
    height=15,
    width=85,
    bg="#e5e7eb",  # Cool gray-200
    fg="#111827",
    relief="solid",
    borderwidth=1
)
log_box.pack(pady=20)

root.mainloop()

Enter your email password:  ········
